In [ ]:
# Trying to find algorithmic methods to find topics
persuasiveness_score = 4

print(sorted_df[sorted_df["persuasiveness_metric"]==persuasiveness_score].shape)
texts = sorted_df[sorted_df["persuasiveness_metric"]==persuasiveness_score]["claim"]
for i, text in enumerate(texts):
    print(i, text)

# 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Convert texts to a document-term matrix
vectorizer = CountVectorizer(stop_words='english')
dtm = vectorizer.fit_transform(texts)

# Fit the LDA model
num_topics = 6
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

num_top_words = 5
tf_feature_names = vectorizer.get_feature_names_out()
display_topics(lda, tf_feature_names, num_top_words)

# Assign topics to texts
topic_assignments = lda.transform(dtm).argmax(axis=1)
texts_with_topics = pd.DataFrame({'Text': texts, 'Topic': topic_assignments})
#print(texts_with_topics)

from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from transformers import pipeline
import nbformat

# Fetch sample data
#data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes')).data
data = texts

# Initialize a transformer-based pipeline for embeddings
embedding_model = pipeline("feature-extraction", model="distilbert-base-uncased", device="mps")

# Initialize BERTopic with the embedding model
topic_model = BERTopic(embedding_model=embedding_model, nr_topics=28)

# Fit the model
topics, probabilities = topic_model.fit_transform(data)

# Display topics
print(topic_model.get_topic_info())

# Visualize topics
topic_model.visualize_topics()
